<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/papers/Artigo%201/V5/CLinsmote/Vers%C3%A3o_5_(CLinsmote_vers%C3%A3o_4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Machine learning for predicting liver and/or lung metastasis in colorectal cancer: A retrospective study based on the SEER database***

Este estudo propõe um modelo de aprendizado de máquina para prever o risco de metástase hepática e/ou pulmonar em pacientes com câncer colorretal (CRC). A partir da base de dados SEER, foram extraídos dados aproximadamente 53 mil pacientes com diagnóstico patológico de CRC entre 2010 e 2015, desenvolvendo sete modelos de algoritmos(Decision tree, Randon Forest, Naive Bayes,  KNN,XGBoost, Gradient Boosting.

### Parte 1:  Importação das Bibliotecas e Carregamento do Dataset

Nesta etapa, importamos as bibliotecas necessárias para análise e carregamos o dataset. Realizamos uma verificação inicial para identificar e remover valores faltantes e definimos as variáveis preditoras (X) e as variáveis alvo (y), preparando os dados para o pré-processamento e a modelagem.

In [ ]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import numpy as np
import pandas as pd


# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/dataset/refs/heads/main/export.csv')
df.dropna(inplace=True)

# Definir as variáveis preditoras e a variável alvo
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis']]

y_liver = df['SEER Combined Mets at DX-liver (2010+)']
y_lung = df['SEER Combined Mets at DX-lung (2010+)']

y = pd.concat([y_liver, y_lung], axis=1)

###Parte 2:  Preparação das Variáveis Alvo e Codificação de Variáveis Categóricas

Nesta etapa, preparamos as variáveis alvo (y), combinando as informações de metástase hepática e pulmonar em uma coluna binária para indicar a presença de metástase. Também aplicamos LabelEncoder para transformar variáveis categóricas de X em valores numéricos, facilitando o uso dos dados em modelos de aprendizado de máquina.

In [ ]:
y = pd.concat([y_liver, y_lung], axis=1)

# Função para combinar as informações de metástase hepática e pulmonar em uma coluna binária 'Binary Mets'
def combine_mets_binary(row):
    if row['SEER Combined Mets at DX-liver (2010+)'] == 'Yes' or row['SEER Combined Mets at DX-lung (2010+)'] == 'Yes':
        return 1  # Com metástase
    else:
        return 0  # Sem metástase

# Aplicar a função para criar a nova coluna binária 'Binary Mets' em 'y'
y['Binary Mets'] = y.apply(combine_mets_binary, axis=1)

# Verificar se 'X' e 'y' têm o mesmo número de amostras
print(f"Tamanho de X: {len(X)}")
print(f"Tamanho de y: {len(y)}")

# Salvar o DataFrame 'y' em um arquivo CSV para referência futura ou análise adicional
y.to_csv('/content/Y.csv')

Tamanho de X: 53448
Tamanho de y: 53448


###Parte 3: Definição e Configuração dos Modelos de Aprendizado de Máquina e Validação Cruzada

Aqui, configuramos os principais algoritmos de aprendizado de máquina, incluindo Decision Tree, Random Forest, SVM, Naive Bayes, KNN, XGBoost e Gradient Boosting. Cada modelo é definido com parâmetros específicos para otimizar o desempenho. Em seguida, aplicamos uma validação cruzada estratificada com 5 divisões para avaliar e comparar a performance dos modelos de maneira consistente e robusta.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

def clin_smote_balance(X, y):
    # Imputação de valores ausentes apenas para colunas numéricas
    num_cols = X.select_dtypes(include=[np.number]).columns
    imputer = SimpleImputer(strategy='mean')
    X[num_cols] = imputer.fit_transform(X[num_cols])

    # Separação das classes
    X['Binary Mets'] = y
    X_major = X[X['Binary Mets'] == 0].drop(columns=['Binary Mets'])
    X_minor = X[X['Binary Mets'] == 1].drop(columns=['Binary Mets'])

    # Perfil típico da classe majoritária via KMeans para numéricas
    num_cols = X_major.select_dtypes(include=[np.number]).columns

    # 1. Padroniza as variáveis numéricas
    scaler = StandardScaler()
    X_major_num_scaled = scaler.fit_transform(X_major[num_cols])

    # 2. Aplica KMeans para identificar subgrupos dentro da classe majoritária
    n_clusters = 15
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X_major_num_scaled)

    # 3. Determina os perfis típicos para cada cluster
    perfil_majoritario = []

    for cluster_label in range(n_clusters):
        perfil_cluster = {}
        cluster_indices = (kmeans.labels_ == cluster_label)
        X_cluster = X_major[cluster_indices]

        # Perfil para variáveis numéricas
        for col in num_cols:
            perfil_cluster[col] = X_cluster[col].median()

        # Perfil para variáveis categóricas
        for col in X_major.columns:
            if col not in num_cols:
                if X_cluster[col].nunique() < 10:
                    freq = X_cluster[col].value_counts(normalize=True)
                    dominante = freq[freq >= 0.7]
                    if not dominante.empty:
                        perfil_cluster[col] = dominante.index[0]
                    else:
                        perfil_cluster[col] = X_cluster[col].mode()[0]
                else:
                    perfil_cluster[col] = X_cluster[col].mode()[0]

        perfil_majoritario.append(perfil_cluster)

    minor_status = []
    # Classificação das amostras minoritárias
    for _, row in X_minor.iterrows():
        match, total = 0, 0

        # --- NUMÉRICAS: compara com centroide mais próximo
        row_scaled = scaler.transform([row[num_cols].values])[0]
        dists = np.linalg.norm(kmeans.cluster_centers_ - row_scaled, axis=1)
        cluster_idx = np.argmin(dists)  # índice do cluster mais próximo
        centroide_prox = kmeans.cluster_centers_[cluster_idx]

        # Comparação numéricas
        for i, col in enumerate(num_cols):
            total += 1
            if abs(row_scaled[i] - centroide_prox[i]) < 0.1:
                match += 1

        # Comparação categóricas (usa o perfil do cluster correspondente)
        perfil_categorico = perfil_majoritario[cluster_idx]
        for col in perfil_categorico:
            if col not in num_cols:
                total += 1
                if row[col] == perfil_categorico[col]:
                    match += 1

        percent = match / total if total > 0 else 0
        minor_status.append('🟢' if percent < 0.3 else '🟡' if percent < 0.7 else '🔴')


    X_minor['Status'] = minor_status


    # Supondo que X_minor já tem a coluna 'Status' com os rótulos 🟢🟡🔴

    # 1. Seleciona todos os verdes
    X_minor_green = X_minor[X_minor['Status'] == '🟢'].drop(columns='Status')

    # 1a. Remove outliers numéricos do grupo verde usando IQR
    def remove_outliers_iqr(df):
        mask = pd.Series([True]*len(df), index=df.index)
        num_cols = df.select_dtypes(include=[np.number]).columns
        for col in num_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            mask = mask & df[col].between(lower, upper)
        return df[mask]

    X_minor_green = remove_outliers_iqr(X_minor_green)


    # 2. Seleciona uma fração dos intermediários (por exemplo, 60%)
    frac_intermediarios = 0.4
    X_minor_yellow = X_minor[X_minor['Status'] == '🟡'].sample(frac=frac_intermediarios, random_state=42).drop(columns='Status')

    # 3. Junta verdes e os intermediários escolhidos
    X_pool_sintetico = pd.concat([X_minor_green, X_minor_yellow])

    # 4. Gera as amostras sintéticas a partir desse pool
    def gerar_sinteticas(df, n_amostras):
        sinteticas = []
        for _ in range(n_amostras):
            base = df.sample(2, replace=True)
            nova = {col: np.random.choice(base[col].values) for col in df.columns}
            sinteticas.append(nova)
        return pd.DataFrame(sinteticas)

    n_sinteticas = max(0, int(0.3 * (len(X_major) - len(X_minor))))
    X_sinteticas = gerar_sinteticas(X_pool_sintetico, n_sinteticas)

    # 5. Junta tudo
    X_major['Binary Mets'] = 0
    X_minor['Binary Mets'] = 1
    X_sinteticas['Binary Mets'] = 1

    X_balanceado = pd.concat([X_major, X_minor, X_sinteticas], ignore_index=True)
    y_balanceado = X_balanceado['Binary Mets']
    X_balanceado.drop(columns=['Binary Mets', 'Status'], errors='ignore', inplace=True)

    return X_balanceado, y_balanceado



In [ ]:
import lightgbm as lgb

cat_cols = X.select_dtypes(include='object').columns
X[cat_cols] = X[cat_cols].astype('category')

# Defina seus modelos:
models = {
    "LightGBM": lgb.LGBMClassifier()
    # ... outros modelos se desejar (mas só LightGBM aceita direto!)
}

smote_techniques = {
    "clin_smote_balance": clin_smote_balance
}


# Configuração da validação cruzada estratificada com 5 divisões (folds)
# Isso garante que a proporção de classes seja mantida em cada divisão, e o shuffle embaralha os dados antes de dividir
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

/tmp/ipython-input-4-3939197387.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = X[cat_cols].astype('category')


### Parte 4: Avaliação e Comparação dos Modelos de Aprendizado de Máquina em Conjuntos de Treino, Validação e Teste


Este bloco de código implementa a validação cruzada para treinar e avaliar os modelos de aprendizado de máquina definidos no pipeline. Ele utiliza a técnica de K-Fold Cross-Validation para dividir os dados em múltiplos folds, garantindo uma avaliação robusta do desempenho dos modelos. Durante cada fold, os dados de treinamento são balanceados utilizando o SMOTE e escalados com o StandardScaler. Métricas de desempenho, como precisão, recall, F1-Score, especificidade, AUC-ROC e AUPR, são calculadas tanto para o conjunto de treinamento quanto para o conjunto de teste. Além disso, visualizações como matrizes de confusão e curvas ROC e Precisão-Recall são geradas. Ao final, as métricas médias de todos os folds são compiladas para comparação.

In [ ]:
import re
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 300, 500],     # tente menos, aumente só se precisar
    'learning_rate': [0.01, 0.05, 0.1], # valores intermediários
    'num_leaves': [15, 31, 63],         # não exagere no começo!
    'max_depth': [5, 7, 9],             # evite -1 por ora
    'min_child_samples': [10, 20],      # não ponha valores altos demais
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


roc_curves_test = {smote_name: {} for smote_name in smote_techniques.keys()}
roc_curves_train = {smote_name: {} for smote_name in smote_techniques.keys()}
results_table_test = []
results_table_train = []

for smote_name, smote in smote_techniques.items():
    for model_name, model in models.items():
        print(f"\nAplicando {smote_name} com {model_name}")
        mean_fpr = np.linspace(0, 1, 100)
        tprs_test, aucs_test = [], []
        accuracies_test, precisions_test, recalls_test, f1_scores_test = [], [], [], []
        tprs_train, aucs_train = [], []
        accuracies_train, precisions_train, recalls_train, f1_scores_train = [], [], [], []

        for train_index, test_index in kf.split(X, y['Binary Mets']):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y['Binary Mets'].iloc[train_index], y['Binary Mets'].iloc[test_index]

            # Balanceamento com RandomOverSampler (funciona com categóricos)
            if smote_name == 'clin_smote_balance':
                X_train_res, y_train_res = smote(X_train.copy(), y_train.copy())
            else:
                X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

            for col in X_train_res.select_dtypes(include='object').columns:
                X_train_res[col] = X_train_res[col].astype('category')
            for col in X_test.select_dtypes(include='object').columns:
                X_test[col] = X_test[col].astype('category')

            def clean_column_names(df):
                df.columns = [
                    re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in df.columns
                ]
                return df

            # Limpar nomes das colunas dos datasets usados no fit e predict
            X_train_res = clean_column_names(X_train_res)
            X_test = clean_column_names(X_test)

            # Treinamento e avaliação
            # Só faça GridSearch para o LightGBM
            if model_name == "LightGBM":
                gs = GridSearchCV(model, param_grid, scoring='f1', cv=3, n_jobs=-1)
                gs.fit(X_train_res, y_train_res, categorical_feature='auto')
                best_model = gs.best_estimator_
                if model_name == "LightGBM":
                    gs = GridSearchCV(model, param_grid, scoring='f1', cv=3, n_jobs=-1)
                    gs.fit(X_train_res, y_train_res, categorical_feature='auto')
                    best_model = gs.best_estimator_
                    print(f"Melhores parâmetros LightGBM nesta rodada: {gs.best_params_}")
                else:
                    best_model = model
                    best_model.fit(X_train_res, y_train_res)

            else:
                best_model = model
                best_model.fit(X_train_res, y_train_res)

            y_pred_test = best_model.predict(X_test)
            y_pred_proba_test = best_model.predict_proba(X_test)[:, 1]

            fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_proba_test)
            interp_tpr_test = np.interp(mean_fpr, fpr_test, tpr_test)
            interp_tpr_test[0] = 0.0
            tprs_test.append(interp_tpr_test)
            aucs_test.append(auc(fpr_test, tpr_test))
            accuracies_test.append(accuracy_score(y_test, y_pred_test))
            precisions_test.append(precision_score(y_test, y_pred_test))
            recalls_test.append(recall_score(y_test, y_pred_test))
            f1_scores_test.append(f1_score(y_test, y_pred_test))

            y_pred_train = best_model.predict(X_train_res)
            y_pred_proba_train = best_model.predict_proba(X_train_res)[:, 1]

            fpr_train, tpr_train, _ = roc_curve(y_train_res, y_pred_proba_train)
            interp_tpr_train = np.interp(mean_fpr, fpr_train, tpr_train)
            interp_tpr_train[0] = 0.0
            tprs_train.append(interp_tpr_train)
            aucs_train.append(auc(fpr_train, tpr_train))
            accuracies_train.append(accuracy_score(y_train_res, y_pred_train))
            precisions_train.append(precision_score(y_train_res, y_pred_train))
            recalls_train.append(recall_score(y_train_res, y_pred_train))
            f1_scores_train.append(f1_score(y_train_res, y_pred_train))

        mean_accuracy_test = np.mean(accuracies_test)
        mean_auc_test = np.mean(aucs_test)
        mean_precision_test = np.mean(precisions_test)
        mean_recall_test = np.mean(recalls_test)
        mean_f1_test = np.mean(f1_scores_test)
        mean_accuracy_train = np.mean(accuracies_train)
        mean_auc_train = np.mean(aucs_train)
        mean_precision_train = np.mean(precisions_train)
        mean_recall_train = np.mean(recalls_train)
        mean_f1_train = np.mean(f1_scores_train)

        print(f"Teste - Accuracy: {mean_accuracy_test:.4f}, AUC: {mean_auc_test:.4f}, Precision: {mean_precision_test:.4f}, Recall: {mean_recall_test:.4f}, F1-score: {mean_f1_test:.4f}")
        print(f"Treinamento - Accuracy: {mean_accuracy_train:.4f}, AUC: {mean_auc_train:.4f}, Precision: {mean_precision_train:.4f}, Recall: {mean_recall_train:.4f}, F1-score: {mean_f1_train:.4f}")

        results_table_test.append({
            "SMOTE Technique": smote_name,
            "Model": model_name,
            "Accuracy": mean_accuracy_test,
            "AUC": mean_auc_test,
            "Precision": mean_precision_test,
            "Recall rate": mean_recall_test,
            "F1-score": mean_f1_test
        })
        results_table_train.append({
            "SMOTE Technique": smote_name,
            "Model": model_name,
            "Accuracy": mean_accuracy_train,
            "AUC": mean_auc_train,
            "Precision": mean_precision_train,
            "Recall rate": mean_recall_train,
            "F1-score": mean_f1_train
        })



Aplicando clin_smote_balance com LightGBM
[LightGBM] [Info] Number of positive: 15357, number of negative: 36433
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 278
[LightGBM] [Info] Number of data points in the train set: 51790, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.296524 -> initscore=-0.863904
[LightGBM] [Info] Start training from score -0.863904
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 15357, number of negative: 36433
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.00724